In [ ]:
from inventory_simulator import InventorySimulator
import numpy as np
from scipy import stats

In [ ]:
def format_percentage(input_ratio):
    """Takes an input ratio, formats it as percentage and caps it between 0 and 100%"""
    input_ratio_capped = min(1, input_ratio)
    input_ratio_capped = max(0, input_ratio_capped)
    rounded_value = round(input_ratio_capped * 100,2)
    return f"{rounded_value} %"

## Run a Single Simulation

In [ ]:
sim = InventorySimulator(yearly_demand_mean=100000,
                         yearly_demand_deviation=200000,
                         vendor_lead_time_mean=4,
                         vendor_lead_time_deviation=0.8,
                         single_transaction_fixed_cost=50,
                         variable_transaction_cost=0.25,
                         holding_cost=0.1,
                         stockout_cost=0.1,
                         review_periods=[7],
                         min_order_size=50,
                         confidence_level=0.95)

In [ ]:
# Simulate
days_to_simulate = 180
sim.simulate(days_to_simulate)

In [ ]:
# Show the simulation results as a df
sim.show_sim_results_as_df().head()

In [ ]:
# Based on the current simulation results plots the inventory and stock outs
sim.plot_results()

In [ ]:
fill_rate, total_service_level, daily_stock_out = sim.calculate_metrics()

# Print variable names and their values
print("fill_rate:", format_percentage(fill_rate))
print("total_service_level:", format_percentage(total_service_level))
print("daily_stock_out:", format_percentage(daily_stock_out))

# Run N simulations

In [ ]:
# Run simulation n times
num_simulations = 10
num_parameters = 3

# Empty array to store results
results = np.empty((num_simulations, num_parameters))

# Run simulations, and store metrics
for i in range(num_simulations):

    sim.simulate(days_to_simulate)
    # Run your simulation and obtain parameters a, b, c
    result_metrics = sim.calculate_metrics()

    # Store the results in the corresponding row of the array
    results[i] = [*result_metrics]


In [ ]:
confidence_intervals = []

for i in range(results.shape[1]):
    c_i = stats.t.interval(sim.confidence_level, len(results[i]) - 1, loc=np.mean(results[i]), scale=stats.sem(results[i]))
    confidence_intervals.append(c_i)

# Print variable names and their values
print("fill_rate between:",format_percentage(confidence_intervals[0][0]), "and",format_percentage(confidence_intervals[0][1]))
print("total_service between:",format_percentage(confidence_intervals[1][0]), "and",format_percentage(confidence_intervals[1][1]))
print("daily_stock_out between:",format_percentage(confidence_intervals[2][0]), "and",format_percentage(confidence_intervals[2][1]))